In [ ]:
import polars as pl
import networkx as nx
from math import sqrt
import random
import rtsvg
rt = rtsvg.RACETrack()

# Create a graph
from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()
g          = _patterns_.createPattern('mesh')
#g          = nx.Graph()
#g.add_edge('a', 'b', weight=1.0), g.add_edge('a', 'c', weight=1.0), g.add_edge('b', 'c', weight=1.0), g.add_edge('c','d', weight=1.0), g.add_edge('d','e', weight=1.0)

# Create the distance dataframe from the graph
_lu_    = {'fm':[], 'to':[], 't':[], 's':[]}
for node in g.nodes:
    for nbor in g.neighbors(node):
        _w_ = g[node][nbor]['weight'] if 'weight' in g[node][nbor] else 1.0
        _lu_['fm'].append(node), _lu_['to'].append(nbor), _lu_['t'].append(_w_), _lu_['s'].append(False)
df_dist = pl.DataFrame(_lu_)

# Create positional dataframe w/ random values
_lu_    = {'node':[], 'x':[], 'y':[]}
for node in g.nodes: _lu_['node'].append(node), _lu_['x'].append(random.random()), _lu_['y'].append(random.random())
df_pos  = pl.DataFrame(_lu_)

# Run the spring simulation
positions = []
k, mu     = 2, 1.0/(2.0*len(g.nodes))
__dx__, __dy__ = (pl.col('x') - pl.col('x_right')), (pl.col('y') - pl.col('y_right'))
for i in range(2*len(g.nodes)):
  positions.append(df_pos)
  df_pos = df_pos.join(df_pos, how='cross') \
                 .filter(pl.col('node') != pl.col('node_right')) \
                 .join(df_dist, left_on=['node', 'node_right'], right_on=['fm','to']) \
                 .with_columns((__dx__**2 + __dy__**2).sqrt().alias('d')) \
                 .with_columns(pl.col('t').pow(k).alias('t_k')) \
                 .with_columns(pl.when(pl.col('d') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('d')).alias('d'),
                               pl.when(pl.col('t') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('t')).alias('t')) \
                 .with_columns((pl.col('t')**(2-k)).alias('__prod_1__'),
                               ((2.0*__dx__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('xadd'),
                               ((2.0*__dy__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('yadd'),
                               (((pl.col('t') - pl.col('d'))**2)/(pl.col('t_k'))**k).alias('__prod_2__')) \
                 .group_by(['node','x','y','s']).agg(pl.col('xadd').sum(), pl.col('yadd').sum(), pl.col('__prod_1__').sum(), pl.col('__prod_2__').sum()) \
                 .with_columns(pl.when(pl.col('s')).then(pl.col('x')).otherwise(pl.col('x') - mu * pl.col('xadd')).alias('x'),
                               pl.when(pl.col('s')).then(pl.col('y')).otherwise(pl.col('y') - mu * pl.col('yadd')).alias('y')) \
                 .drop(['xadd','yadd', 's', '__prod_1__', '__prod_2__'])
positions.append(df_pos)

# Make an animation
_dur_ = '5s'
x0, y0, x1, y1 = positions[0]['x'].min(), positions[0]['y'].min(), positions[0]['x'].max(), positions[0]['y'].max()
for _pos_ in positions: x0, y0, x1, y1 = min(x0, _pos_['x'].min()), min(y0, _pos_['y'].min()), max(x1, _pos_['x'].max()), max(y1, _pos_['y'].max())
svg = [f'<svg x="0" y="0" width="256" height="256" viewBox="{x0} {y0} {x1-x0} {y1-y0}">']
svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="white" />')
node_positions = {}
for _node_ in g.nodes: node_positions[_node_] = []
for _pos_ in positions:
    for i in range(len(_pos_)):
        _node_, _x_, _y_ = _pos_['node'][i], _pos_['x'][i], _pos_['y'][i]
        node_positions[_node_].append((_x_, _y_))
for _node_ in g.nodes:
    for _nbor_ in g.neighbors(_node_):
        _x1_, _y1_ = node_positions[_node_][0]
        _x2_, _y2_ = node_positions[_nbor_][0]
        x1_strs, y1_strs = [], []
        for _x_, _y_ in node_positions[_node_]: x1_strs.append(f'{_x_}'), y1_strs.append(f'{_y_}')
        x2_strs, y2_strs = [], []
        for _x_, _y_ in node_positions[_nbor_]: x2_strs.append(f'{_x_}'), y2_strs.append(f'{_y_}')
        svg.append(f'<line x1="{_x1_}" y1="{_y1_}" x2="{_x2_}" y2="{_y2_}" stroke="black" stroke-width="0.002">')
        svg.append(f'<animate attributeName="x1" values="{";".join(x1_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="y1" values="{";".join(y1_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="x2" values="{";".join(x2_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="y2" values="{";".join(y2_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append('</line>')

for _node_ in node_positions:
    _x_, _y_ = node_positions[_node_][0]
    svg.append(f'<circle cx="{_x_}" cy="{_y_}" r="0.01" fill="black">')
    x_strs, y_strs = [], []
    for _x_, _y_ in node_positions[_node_]: x_strs.append(f'{_x_}'), y_strs.append(f'{_y_}')
    svg.append(f'<animate attributeName="cx" values="{";".join(x_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="cy" values="{";".join(y_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</circle>')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
#
# "Graph Drawing by Force-directed Placement"
# Fruchterman & Reingold
# Sofftware -- Practice and Experience, Vol. 21 (1 1), 1129-1164 (November 1991)
#
# Figure 1 (Force-directed Placement)
#
t    = 10.0
W, H = 256, 256
pos  = {}
for _node_ in g.nodes(): pos[_node_] = random.random()*W, random.random()*H
area = W*H
C   = 0.1 # paper says that C is found experimentally...
k   = C*sqrt(area/len(g.nodes()))
def f_a(z): return  z**2 / k # pseudocode says "x" vs "z" ... paper comments say "z"
def f_r(z): return -k**2 / z # paper comments differ on the sign

_iterations_ = 2*len(g.nodes())
for i in range(_iterations_):
    disp = {}
    # calculate repulsive forces
    for v in g.nodes():
        disp[v] = 0.0, 0.0
        for u in g.nodes():
            if u == v or g.has_edge(u, v): continue
            _diff_     = pos[v][0] - pos[u][0], pos[v][1] - pos[u][1]
            _diff_len_ = sqrt(_diff_[0]**2 + _diff_[1]**2)
            if _diff_len_ < 0.001: _diff_len_ = 0.001
            disp[v]    = disp[v][0] + (_diff_[0]/_diff_len_) * f_r(_diff_len_), disp[v][1] + (_diff_[1]/_diff_len_) * f_r(_diff_len_)
    # calculate the attractive forces
    for e in g.edges():
        if e[0] < e[1]: # process each edge only once... "each edge is an ordered. pair of vertices"
            e_v, e_u = e[0], e[1]
            _diff_     = pos[e_v][0] - pos[e_u][0], pos[e_v][1] - pos[e_u][1]
            _diff_len_ = sqrt(_diff_[0]**2 + _diff_[1]**2)
            if _diff_len_ < 0.001: _diff_len_ = 0.001
            disp[e_v]  = disp[e_v][0] - (_diff_[0]/_diff_len_) * f_a(_diff_len_), disp[e_v][1] - (_diff_[1]/_diff_len_) * f_a(_diff_len_)
            disp[e_u]  = disp[e_u][0] + (_diff_[0]/_diff_len_) * f_a(_diff_len_), disp[e_u][1] + (_diff_[1]/_diff_len_) * f_a(_diff_len_)
    # limit the maximum displacement to the temperature t
    # ... and then prevent from being displaced outside frame
    for v in g.nodes():
        disp_len = sqrt(disp[v][0]**2 + disp[v][1]**2)
        pos[v] = pos[v][0] + (disp[v][0]/disp_len)*min(t, disp[v][0]), pos[v][1] + (disp[v][1]/disp_len)*min(t, disp[v][1])
        pos[v] = min(W/2, max(-W/2, pos[v][0])), min(H/2, max(-H/2, pos[v][1]))
    t *= 0.5 # t = cool(t)

_lu_ = {'fm':[], 'to':[]}
for e in g.edges(): _lu_['fm'].append(e[0]), _lu_['to'].append(e[1])
df = pl.DataFrame(_lu_)
rt.link(df, [('fm','to')], pos)